법률 데이터를 텍스트 마이닝할 때 유용한 분석 방법은 주로 텍스트의 구조적 특성, 법률 문서의 맥락, 문서 간의 관계를 이해하는 데 중점을 둡니다. 법률 문서에는 특정한 용어와 구조가 반복적으로 나타나므로 이를 분석하는 방법이 일반적인 텍스트 분석과는 약간 다를 수 있습니다. 

1. 주요 법률 용어 빈도 분석
법률 문서에는 특정 법률 용어나 관련 키워드가 자주 등장합니다. 주요 키워드의 빈도를 분석하여, 사건의 주제나 핵심 법률을 파악할 수 있습니다.

In [ ]:
from konlpy.tag import Okt
from collections import Counter

okt = Okt()

# 법률 문서에서 명사 추출
df['nouns'] = df['text'].apply(lambda x: okt.nouns(x))

# 모든 문서에서 추출한 명사 리스트 생성
all_words = [word for words in df['nouns'] for word in words]

# 단어 빈도 계산
word_freq = Counter(all_words)

# 상위 10개 자주 등장하는 단어 출력
common_words = word_freq.most_common(10)
print("자주 등장하는 법률 용어:", common_words)


2. TF-IDF (Term Frequency - Inverse Document Frequency) 분석
법률 문서에서는 특정 사건에서만 자주 등장하는 단어를 분석하기 위해 TF-IDF를 사용합니다. 이는 문서의 특정 단어가 다른 문서에 비해 얼마나 중요한지를 나타내는 지표입니다.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 한국어 텍스트를 형태소 분석으로 나눈 후 사용
df['tokenized_text'] = df['text'].apply(lambda x: ' '.join(okt.morphs(x)))

# TF-IDF 벡터라이저 적용
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
tfidf_matrix = tfidf_vectorizer.fit_transform(df['tokenized_text'])

# TF-IDF 스코어 확인
terms = tfidf_vectorizer.get_feature_names_out()
print("TF-IDF 용어:", terms[:10])


3. N-그램 분석
법률 문서에서 중요한 문구가 자주 반복되므로, 단일 단어보다 2-그램(바이그램) 또는 3-그램(트라이그램) 분석이 더 유용할 수 있습니다. 이를 통해 자주 사용되는 단어 쌍 또는 구문을 발견할 수 있습니다

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# 2-그램 분석
vectorizer = CountVectorizer(ngram_range=(2, 2))  # 바이그램 분석
X2 = vectorizer.fit_transform(df['tokenized_text'])

# 상위 10개의 자주 등장하는 바이그램 확인
bi_grams = vectorizer.get_feature_names_out()
bi_gram_counts = X2.toarray().sum(axis=0)
bi_gram_freq = dict(zip(bi_grams, bi_gram_counts))
sorted_bi_grams = sorted(bi_gram_freq.items(), key=lambda x: x[1], reverse=True)[:10]
print("상위 10개 바이그램:", sorted_bi_grams)


4. LDA (Latent Dirichlet Allocation)를 통한 주제 모델링
법률 문서의 주제(예: 형사, 민사, 상속, 계약 등)를 자동으로 분류하고자 할 때 LDA를 사용하여 문서가 다루는 주제를 추출할 수 있습니다.

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

# 카운트 벡터라이저를 이용한 텍스트 벡터화
vectorizer = CountVectorizer(max_features=1000, stop_words='english')
X = vectorizer.fit_transform(df['tokenized_text'])

# LDA 모델 적용
lda = LatentDirichletAllocation(n_components=5, random_state=42)
lda.fit(X)

# 각 문서에 대한 주제 분포 확인
topics = lda.transform(X)
print("문서별 주요 주제 분포:", topics[:5])

# 각 주제에 해당하는 상위 단어 출력
for i, topic in enumerate(lda.components_):
    print(f"주제 {i}: {[vectorizer.get_feature_names_out()[i] for i in topic.argsort()[:-10 - 1:-1]]}")


5. 단어간 상관관계 분석
법률 문서에서 특정 단어들이 함께 사용되는 경우가 많습니다. 이를 분석해 단어들 간의 상관관계를 파악하여, 문서 내 연관성을 분석할 수 있습니다.

import numpy as np
import seaborn as sns

# 특정 법률 용어들의 상관관계를 분석할 단어 리스트
words_to_check = ['법률', '판례', '형법', '민법']

# 각 단어가 문서에서 얼마나 자주 등장하는지 카운트
word_matrix = np.zeros((len(df), len(words_to_check)))

for i, text in enumerate(df['text']):
    for j, word in enumerate(words_to_check):
        word_matrix[i, j] = text.count(word)

# 단어 간 상관관계 계산
correlation_matrix = np.corrcoef(word_matrix.T)

# 히트맵으로 상관관계 시각화
sns.heatmap(correlation_matrix, annot=True, xticklabels=words_to_check, yticklabels=words_to_check)
plt.title('단어 간 상관관계')
plt.show()


6. Word2Vec을 이용한 단어 임베딩
법률 문서에서 특정 단어들이 서로 어떻게 연결되어 있는지를 분석하는 데 Word2Vec 같은 단어 임베딩 모델이 유용합니다. 단어 간 유사성을 측정할 수 있으며, 이를 통해 문서의 구조를 파악할 수 있습니다.

In [ ]:
from gensim.models import Word2Vec
from konlpy.tag import Okt

okt = Okt()

# 문서 토큰화 (명사만 추출)
tokenized_texts = df['text'].apply(lambda x: okt.nouns(x))

# Word2Vec 모델 학습
w2v_model = Word2Vec(sentences=tokenized_texts, vector_size=100, window=5, min_count=5, workers=4)

# '법률'과 가장 유사한 단어 찾기
print(w2v_model.wv.most_similar('법률', topn=10))


7. 네트워크 분석 (단어-법률, 판사-변호사 관계)
법률 문서에서 단어들 간의 관계 또는 판사와 변호사 간의 관계를 네트워크로 시각화할 수 있습니다. 법률 문서에서 누가 참조되었는지, 어떤 법이 자주 사용되었는지 등의 관계를 네트워크로 분석할 수 있습니다.

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

# 샘플 데이터: 판사와 변호사 간의 관계
edges = [('판사A', '변호사X'), ('판사A', '변호사Y'), ('판사B', '변호사X'), ('판사B', '변호사Z')]

# 네트워크 그래프 생성
G = nx.Graph()
G.add_edges_from(edges)

# 네트워크 시각화
plt.figure(figsize=(8, 6))
nx.draw(G, with_labels=True, node_color='skyblue', font_weight='bold', node_size=3000)
plt.title('판사-변호사 네트워크')
plt.show()


8. 감정 분석 (법률 문서의 긍정/부정 감정 분류)
법률 문서에서 특정 문장의 긍정적/부정적 성향을 분석해 사건의 결과를 예측하거나 판례의 특성을 분석할 수 있습니다

In [ ]:
from konlpy.tag import Okt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

okt = Okt()

# 감정 분류를 위한 데이터 전처리 (예: 긍정/부정 레이블이 포함된 데이터)
df['tokenized_text'] = df['text'].apply(lambda x: ' '.join(okt.morphs(x)))
X_train, X_test, y_train, y_test = train_test_split(df['tokenized_text'], df['label'], test_size=0.2, random_state=42)

# TF-IDF 벡터화 및 로지스틱 회귀 모델 사용
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

# 모델 평가
accuracy = model.score(X_test_tfidf, y_test)
print(f"감정 분석 정확도: {accuracy:.2f}")
``
